<a href="https://colab.research.google.com/github/ShuqairABD/DenseNet-VGG-/blob/main/DenseNet_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **start**

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
from torch.utils.data import Dataset, Subset, DataLoader
from torchvision import models, transforms
from torchvision.io import read_image, ImageReadMode
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import albumentations as A
from torchvision.transforms import v2
import numpy as np
import cv2
from PIL import Image
import PIL
from pathlib import Path
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
from torchsummary import summary
from datetime import datetime
from tqdm import tqdm
import gc
import random
import pandas as pd
import pprint
from tabulate import tabulate

import torchvision.models as models
from torch.optim import Adam
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset


from torchvision import models, transforms, datasets



In [ ]:

from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

import time
import torch.optim as optim

# **VGG**

In [ ]:
# Путь к папке
data_dir = '/content/drive/MyDrive/WEATHER'

# преобразования для данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загрузка данные
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Разделить данные на обучающую и валидационную выборки
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42, stratify=dataset.targets)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Загрузка предобученную модель VGG16
model = models.vgg16(pretrained=True)

# Изменяем последний слой для классификации на два класса
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# критерий потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:10<00:00, 52.0MB/s]


In [ ]:
# Функция для обучения модели
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25, save_path='model.pth'):
    start_time = time.time()
    print("Training started")
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Training Loss: {epoch_loss:.4f}')

        model.eval()
        val_loss = 0.0
        corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

        # сохранение наилучшей модели
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print(f'Training completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s')
    print(f'Best validation accuracy: {best_acc:.4f}')

    # Загрузка лучших весов модели
    model.load_state_dict(best_model_wts)

    # Сохранение модели
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

# Обучаем модель
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10, save_path='VGG_model.pth')


Training started
Epoch 1/10
Training Loss: 0.0806
Validation Loss: 0.0650, Accuracy: 0.9763
Epoch 2/10
Training Loss: 0.0262
Validation Loss: 0.0419, Accuracy: 0.9858
Epoch 3/10
Training Loss: 0.0171
Validation Loss: 0.0332, Accuracy: 0.9873
Epoch 4/10
Training Loss: 0.0109
Validation Loss: 0.0419, Accuracy: 0.9873
Epoch 5/10
Training Loss: 0.0093
Validation Loss: 0.0327, Accuracy: 0.9896
Epoch 6/10
Training Loss: 0.0057
Validation Loss: 0.0463, Accuracy: 0.9887
Epoch 7/10
Training Loss: 0.0037
Validation Loss: 0.0372, Accuracy: 0.9893
Epoch 8/10
Training Loss: 0.0031
Validation Loss: 0.0427, Accuracy: 0.9899
Epoch 9/10
Training Loss: 0.0044
Validation Loss: 0.0487, Accuracy: 0.9873
Epoch 10/10
Training Loss: 0.0031
Validation Loss: 0.0379, Accuracy: 0.9887
Training completed in 64m 59s
Best validation accuracy: 0.9899
Model saved to VGG_model.pth


## **test VGG**

In [ ]:


# Путь к папке (test)
test_data_dir = '/content/drive/MyDrive/test_images'

#преобразования для данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# предобученной модели VGG16
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Linear(num_ftrs, 2)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Загрузка обученной модели
model.load_state_dict(torch.load('/content/VGG_model.pth'))

# отображения изображения
def imshow(inp, title=None):
    """Denormalize and show an image"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

# Функция для тестирования модели
def test_model(model, data_dir, class_names):
    model.eval()
    images = []
    image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image = data_transforms(image)
        images.append((image, image_path))

    with torch.no_grad():
        for image, image_path in images:
            input_tensor = image.unsqueeze(0).to(device)
            outputs = model(input_tensor)
            _, preds = torch.max(outputs, 1)

            # Визуализация изображения и предсказания
            plt.figure()
            imshow(image.cpu(), title=f'Predicted: {class_names[preds[0]]}')
            plt.show()

# имена классов
class_names = ['cold_season', 'warm_season']

# Тест модель
test_model(model, test_data_dir, class_names)


### **VGG sigmoid**

In [ ]:

# Путь к папке с изображениями
data_dir = '/content/drive/MyDrive/WEATHER'

# преобразования данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загружаем данные
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Разделяем данные
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42, stratify=dataset.targets)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

#  модель VGG16
model = models.vgg16(pretrained=True)

# Изменяем последний слой для бинарной классификации
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# потерь и оптимизатор
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Функция для обучения модели
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        model.eval()
        val_loss = 0.0
        corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                preds = (outputs > 0.5).float()
                corrects += torch.sum(preds == labels.data)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

        # Сохранение модели
        torch.save(model.state_dict(), f'vgg_model_epoch_{epoch+1}.pth')

# Обучаем модель
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=5)


**test VGG sigmoid**

In [ ]:

# Путь к папке с тестовыми изображениями
test_data_dir = '/content/drive/MyDrive/test_images'

# Определяем преобразования для данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загрузка предобученной модели VGG16 и изменение последнего слоя
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = torch.nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Перемещаем модель на GPU, если доступно
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Загрузка обученной модели
model.load_state_dict(torch.load('vgg_model_epoch_5.pth'))

# Функция для отображения изображения
def imshow(inp, title=None):
    """Denormalize and show an image"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

# Функция для тестирования модели
def test_model(model, data_dir, class_names):
    model.eval()
    images = []
    image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image = data_transforms(image)
        images.append((image, image_path))

    with torch.no_grad():
        for image, image_path in images:
            input_tensor = image.unsqueeze(0).to(device)
            outputs = model(input_tensor)
            preds = (outputs > 0.5).float()

            # Визуализация изображения и предсказания
            plt.figure()
            imshow(image.cpu(), title=f'Predicted: {class_names[int(preds[0].item())]}')
            plt.show()

# Определяем имена классов
class_names = ['cold_season', 'warm_season']

# Тестируем модель и визуализируем предсказания
test_model(model, test_data_dir, class_names)


**save weights to drive**

In [ ]:
# Копирование файла весов в Google Drive
!cp /content/VGG_model.pth /content/drive/MyDrive/VGG_model.pth

# Проверка наличия файла в Google Drive
!ls /content/drive/MyDrive/

# Проверка размера файла в Google Drive
!ls -lh /content/VGG_model.pth

# Проверка размера исходного файла в Colab
!ls -lh /content/VGG_model.pth

# **DenseNet**

In [ ]:

# Путь к папке
data_dir = '/content/drive/MyDrive/WEATHER'

# преобразования для данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загрузка данные
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# Разделить данные на обучающую и валидационную выборки
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42, stratify=dataset.targets)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# предобученную модель DenseNet121
model = models.densenet121(pretrained=True)

# Изменяем последний слой для классификации на два класса
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# критерия потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.9)

# Функция для обучения модели
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=3, save_path='model.pth'):
    start_time = time.time()
    print("Training started")
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Training Loss: {epoch_loss:.4f}')

        model.eval()
        val_loss = 0.0
        corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

        # Сохранение наилучшей модели
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print(f'Training completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s')
    print(f'Best validation accuracy: {best_acc:.4f}')

    # Загрузка лучших весов модели
    model.load_state_dict(best_model_wts)

    # Сохранение модели
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

# Обучаем модель
num_epochs = 25
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=num_epochs, save_path='densenet_model_2.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training started
Epoch 1/25
Training Loss: 0.0961
Validation Loss: 0.0356, Accuracy: 0.9861
Epoch 2/25
Training Loss: 0.0377
Validation Loss: 0.0387, Accuracy: 0.9858
Epoch 3/25
Training Loss: 0.0168
Validation Loss: 0.0303, Accuracy: 0.9884
Epoch 4/25
Training Loss: 0.0095
Validation Loss: 0.0298, Accuracy: 0.9905
Epoch 5/25
Training Loss: 0.0072
Validation Loss: 0.0363, Accuracy: 0.9882
Epoch 6/25
Training Loss: 0.0058
Validation Loss: 0.0305, Accuracy: 0.9902
Epoch 7/25
Training Loss: 0.0049
Validation Loss: 0.0355, Accuracy: 0.9873
Epoch 8/25
Training Loss: 0.0049
Validation Loss: 0.0365, Accuracy: 0.9899
Epoch 9/25
Training Loss: 0.0042
Validation Loss: 0.0325, Accuracy: 0.9896
Epoch 10/25
Training Loss: 0.0030
Validation Loss: 0.0318, Accuracy: 0.9905
Epoch 11/25
Training Loss: 0.0030
Validation Loss: 0.0319, Accuracy: 0.9902
Epoch 12/25
Training Loss: 0.0028
Validation Loss: 0.0361, Accuracy: 0.9905
Epoch 13/25
Training Loss: 0.0026
Validation Loss: 0.0337, Accuracy: 0.9902
Epoc

In [ ]:
# Копирование файла весов в Google Drive
!cp /content/densenet_model_epoch_5.pth /content/drive/MyDrive/densenet_model_epoch_5.pth

# Проверка наличия файла в Google Drive
!ls /content/drive/MyDrive/

# Проверка размера файла в Google Drive
!ls -lh /content/densenet_model_epoch_5.pth

# Проверка размера исходного файла в Colab
!ls -lh /content/densenet_model_epoch_5.pth

## **test DenseNet**

In [ ]:

# Путь к папке
test_data_dir = '/content/drive/MyDrive/test_images'

# преобразования данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загрузка предобученную модель DenseNet121
model = models.densenet121(pretrained=True)

# Изменяем последний слой для классификации на два класса
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Загрузка обучанной модели
model.load_state_dict(torch.load('/content/densenet_model_2.pth'))

# Функция для отображения изображения
def imshow(inp, title=None):
    """Denormalize and show an image"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

# Функция для тестирования модели
def test_model(model, data_dir, class_names):
    model.eval()
    images = []
    image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image = data_transforms(image)
        images.append((image, image_path))

    with torch.no_grad():
        for image, image_path in images:
            input_tensor = image.unsqueeze(0).to(device)
            outputs = model(input_tensor)
            _, preds = torch.max(outputs, 1)

            # изображения и предсказания
            plt.figure()
            imshow(image.cpu(), title=f'Predicted: {class_names[preds[0]]}')
            plt.show()

# имена классов
class_names = ['cold_season', 'warm_season']

# Тест модель
test_model(model, test_data_dir, class_names)


### **DenseNet sigmoid**

In [ ]:

# Путь
data_dir = '/content/drive/MyDrive/WEATHER'


data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Загружаем данные
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)

# обучающую и валидационную выборки
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42, stratify=dataset.targets)
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# предобученную модель DenseNet121
model = models.densenet121(pretrained=True)

# последний слой для бинарной классификации
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# критерий потерь и оптимизатор
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Функция для обучения модели
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        model.eval()
        val_loss = 0.0
        corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                preds = (outputs > 0.5).float()
                corrects += torch.sum(preds == labels.data)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = corrects.double() / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

        # Сохранение модель
        torch.save(model.state_dict(), f'densenet_model_epoch_{epoch+1}.pth')

# Обучаем модель
train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=5)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5, Loss: 0.1301
Validation Loss: 0.0485, Accuracy: 0.9827
Epoch 2/5, Loss: 0.0434
Validation Loss: 0.0370, Accuracy: 0.9876
Epoch 3/5, Loss: 0.0253
Validation Loss: 0.0343, Accuracy: 0.9884
Epoch 4/5, Loss: 0.0142
Validation Loss: 0.0342, Accuracy: 0.9884
Epoch 5/5, Loss: 0.0133
Validation Loss: 0.0320, Accuracy: 0.9908


### **test DenseNet sigmoid**

In [ ]:

# Путь к папке
test_data_dir = '/content/drive/MyDrive/test_images'

# преобразования для данных
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# DenseNet121
model = models.densenet121(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = torch.nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

# Перемещаем модель на GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# обученной модель
model.load_state_dict(torch.load('/content/densenet_model_epoch_5.pth'))

# Функция для отображения изображения
def imshow(inp, title=None):
    """Denormalize and show an image"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

# Функция для тестирования модели
def test_model(model, data_dir, class_names):
    model.eval()
    images = []
    image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image = data_transforms(image)
        images.append((image, image_path))

    with torch.no_grad():
        for image, image_path in images:
            input_tensor = image.unsqueeze(0).to(device)
            outputs = model(input_tensor)
            preds = (outputs > 0.5).float()

            # изображения и предсказания
            plt.figure()
            imshow(image.cpu(), title=f'Predicted: {class_names[int(preds[0].item())]}')
            plt.show()

# имена классов
class_names = ['cold_season', 'warm_season']

# Тест модель
test_model(model, test_data_dir, class_names)
